In [109]:
import pandas as pd
import re
from eunjeon import Mecab
import collections

In [110]:
dirr = 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/estate_news/'

In [111]:
news_df = pd.read_csv(dirr+'RAA_부동산114뉴스기사_유승훈.csv')
news_df = news_df.dropna()

In [112]:
news_df['yearmonth'] = news_df['NEWS_WRT_YMD'].str.slice(stop=7).str.replace('.','')
news_df

C:\Users\shic\anaconda3\envs\simon_env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


,NEWS_TITLE_NM,NEWS_WRT_YMD,NEWS_MEDIA_NM,NEWS_BODY_CNTT,yearmonth
0,"[주간부동산] 매매·전세, 연말연시 맞아 `한산`",2012.01.01,이데일리,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서...",201201
1,"연말 거래 한산, 매매도 전세도 약세 지속",2012.01.01,조선비즈,서울 아파트 값이 2주 연속 하락세다. 2011년 마지막 주 시장은 비수기로 한산했...,201201
2,"2012년 아파트 공급 줄고, 오피스텔 늘고",2012.01.01,헤럴드POP,"새해에는 브랜드 아파트 공급은 작년보다 줄고, 대표적인 수익형부동산 상품인 오피스텔...",201201
3,수도권 집 한 채에 지방 아파트 몇 채 산다고?,2012.01.01,헤럴드POP,수도권 아파트 한 채 값에 지방 아파트 여러 채를 산다는 말도 옛말이 됐다. 지방 ...,201201
4,고급빌라 맞수 청담동 vs 한남동…“유엔빌리지(한남동) 살아요” ‘70억 펜트하우스...,2012.01.01,매경이코노미,한남동 유엔빌리지. / 청담동 마크힐스. TV 드라마 속 부잣집에서 전화를 받는 사...,201201
...,...,...,...,...,...
45907,[부동산캘린더] 3기 신도시 사전청약 1순위 접수…분양시장 후끈,2021.07.31,연합뉴스,(서울=연합뉴스) 김동규 기자 = 다음 주에는 3기 신도시 사전청약 1순위 접수를 ...,202107
45908,[분양캘린더]8월 첫째 주 1만4368가구 분양…견본주택 3곳 개관,2021.07.31,뉴시스,전국 20개 단지…일반분양 1만86가구[서울=뉴시스] 31일 부동산114에 따르면 ...,202107
45909,올 상반기 청약률 상위 단지는 초등학교 도보 통학권,2021.07.31,데일리안,"분양 시장 주 수요층 3040세대, 자녀 교육 우선안전한 통학 환경 선호해올해 상반...",202107
45910,집값에 치이고 전셋값에 또 치이고,2021.07.31,비즈니스워치,[집값 톡톡]전셋값→집값 상승 악순환서울·수도권·지방까지 전셋값 상승 확산집값은 '...,202107


In [113]:
dates = news_df['yearmonth'].unique().tolist()

In [114]:
list_tmp = []
for dd in dates:
    # empty dictionary
    dict_tmp = {}
    
    # select data by date
    df_tmp = news_df[news_df['yearmonth']==dd]
    
    dict_tmp['dates'] = dd
    content_txt = ' '.join(df_tmp['NEWS_BODY_CNTT'])
    content_txt = re.sub('\([^)]*\)','',content_txt)
    content_txt = re.sub('\[[^)]*\]','',content_txt)
    content_txt = re.sub('\<[^)]*\>','',content_txt)
    content_txt = content_txt.replace('부동산114','')
    dict_tmp['content'] = content_txt
    
    list_tmp.append(pd.DataFrame(dict_tmp,index=[0]))

In [115]:
df_content = pd.concat(list_tmp).reset_index(drop=True)
df_content

,dates,content
0,201201,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서..."
1,201202,투자분석 보고서분담금 9000만원까지 절감1만 가구가 넘는 대단지로 변모하는 서울...
2,201203,"동영상 뉴스【 앵커멘트 】 김포 한강과 파주 운정, 그리고 인천 송도·청라 지구...."
3,201204,▶ 성폭행 가담 男아이돌 멤버 형사처벌 검토▶ ‘K팝’ 이하이·박지민·백아연 3女만...
4,201205,▶ 히로뽕 먹인 여성 실신… ‘엉큼男’ 성추행 미수▶ 이번엔 ‘나체女’… ‘막가는’...
...,...,...
110,202103,"잠실 엘스, 2억 내린 전세매물 등장두달새 25% 증가…서울·경기 비슷“다만 입주 ..."
111,202104,"- 대전 하늘채 스카이앤 2차, 5월중 분양일정 순항… 상담 가능한 ‘사전홍보관’ ..."
112,202105,동영상 뉴스[Copyrightⓒmunhwa.com '대한민국 오후를 여는 유일석간 ...
113,202106,매년 6월 1일 기준 보유세 부과6월부터 매물잠김 시작되는 이유올해는 양도세 중과까...


In [116]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(100)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [117]:
tagger_100 = pd.concat(tagger_list)
tagger_100.to_csv(dirr+'wordcount_df_100.csv',index=False,encoding='UTF-8-sig')

In [118]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(50)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [119]:
tagger_50 = pd.concat(tagger_list)
tagger_50.to_csv(dirr+'wordcount_df_50.csv',index=False,encoding='UTF-8-sig')